# Baseline

This notebook preprocess the images for training, defines and trains a **ResNet50** model, and finally creates a submission.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.callbacks import Callback
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.preprocessing import image
from sklearn.metrics import roc_auc_score

## Read images and labels

In [ ]:
data_path = os.path.join(os.getcwd(), '.', 'input')

In [ ]:
train_path = os.path.join(data_path, 'train')
labels_path = os.path.join(data_path, 'train_labels.csv')

In [ ]:
df = pd.read_csv(labels_path)
df = df.head(1000)  # Remove this line for full data
df.head()

In [ ]:
def crop_roi(img, roi_size=48):
    """Crop a square region in the center of the image."""
    size = img.shape[0]
    roi_ul = (int(size / 2 - roi_size / 2), int(size / 2 - roi_size / 2))
    roi_lr = (int(size / 2 + roi_size / 2), int(size / 2 + roi_size / 2))
    return img[roi_ul[1]: roi_lr[1], roi_ul[0]: roi_lr[0]]

def load_image(img_id, img_size=96, roi_size=None):
    """Load image using its id. Resize and crop is optional."""
    img_path = os.path.join(train_path, '{}.tif'.format(img_id))
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img = image.img_to_array(img)
    if roi_size:
        return crop_roi(img, roi_size)
    return img

In [ ]:
img = load_image(df.id[0])
plt.imshow(img / 255)

## Load and preprocess train set

Let's load all the images and annotations and pickle the results. For future reference the following format is used for pickled names:

`train_images_[img_size]_[roi_size].pkl`

The structure pickled is a numpy array of shape (`N`, `roi_size`, `roi_size`, 3). Where `N` is the size of the train set (220025 in our case), the image channels are in RGB order (Keras' standard) and pixel values aren't normalized (they range from 0 to 255). Images are load from the `.tif` files, resized to `img_size`x`img_size`, and then we crop the center (region of interest) of this image, a square of size `roi_size`x`roi_size`.

In [ ]:
img_size = 96
roi_size = None  # Do not crop center square

In [ ]:
if roi_size is None:
    size = img_size
else:
    size = roi_size

In [ ]:
images = []
labels = []

In [ ]:
for idx, row in df.iterrows():
    img_id, label = row
    img = load_image(img_id, img_size=img_size, roi_size=roi_size)
    img = img.reshape(1, size, size, 3)
    images.append(img)
    labels.append(label)

In [ ]:
images = np.concatenate(images, axis=0)
labels = np.array(labels).reshape(images.shape[0], 1)

In [ ]:
print("images: {}".format(images.shape))
print("labels: {}".format(labels.shape))

## Pickle train data for future usage

Let's save the train set to bu used in other models.

*NOTE*: these images aren't normalized.

In [ ]:
with open(os.path.join(data_path, 'train_images_96_96.pkl'), 'wb') as fout:
    pickle.dump(images, fout, protocol=4)
with open(os.path.join(data_path, 'train_labels.pkl'), 'wb') as fout:
    pickle.dump(labels, fout, protocol=4)

## Modeling

### Normalize images

There are several ways of normalizing pixel values, here we use the method used to train **ResNet50** on *ImageNet* since we are using those pre-trained weights.

In [ ]:
# images = images * 2. / 255. - 1.
images = preprocess_input(images)

### Define callbacks and model architecture

In [ ]:
class RocCallback(Callback):
    """Define a callback which returns train ROC AUC after each epoch."""

    def __init__(self, training_data, validation_data=None):
        self.x = training_data[0]
        self.y = training_data[1]
        self.validation = validation_data is not None
        if self.validation:
            self.x_val = validation_data[0]
            self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        if self.validation:
            y_pred_val = self.model.predict(self.x_val)
            roc_val = roc_auc_score(self.y_val, y_pred_val)
            print('\rroc-auc: {} - roc-auc-val: {}'.format(round(roc, 5), round(roc_val, 5)), end=80 * ' ' + '\n')
        else:
            print('\rroc-auc: {}'.format(round(roc, 5)), end=80 * ' ' + '\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [ ]:
def resnet50():
    resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(size, size, 3), pooling='avg')
    x = resnet.output
    # x = Flatten()(x)
    # x = GlobalAveragePooling2D()(x)
    # x = Dropout(0.5)(last)
    # x = Dense(32, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=[resnet.input], outputs=[x])

In [ ]:
model = resnet50()
model.summary()

### Train model

In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True),
    metrics=['accuracy']
)
callbacks = [RocCallback(training_data=(images, labels))]
model.fit(images, labels, batch_size=16, epochs=20, callbacks=callbacks)

### Save trained model

In [ ]:
model.save('resnet50.h5')

## Test and submission

In [ ]:
test_path = os.path.join(data_path, 'test')
submission_path = os.path.join(data_path, 'sample_submission.csv')

In [ ]:
submission = pd.read_csv(submission_path).drop('label', axis=1)
submission = submission.head(1000)  # Remove this line for full data
submission.head()

In [ ]:
test_images = []
for idx, row in df.iterrows():
    img_id = row[0]
    img = load_image(img_id, img_size=img_size, roi_size=roi_size)
    img = img.reshape(1, size, size, 3)
    test_images.append(img)
test_images = np.concatenate(test_images, axis=0)

In [ ]:
# test_images = test_images * 2. / 255. - 1.
test_images = preprocess_input(test_images)

In [ ]:
model = load_model('resnet50.h5')
predictions = model.predict(test_images).squeeze().tolist()

In [ ]:
submission['label'] = predictions
submission.to_csv('submission.csv', index=False)